<a href="https://colab.research.google.com/github/dtim-upc/THOR/blob/main/UniversalNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import json
from spacy import displacy
from spacy.tokens import Span

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 6.8 MB/s eta 0:00:00


In [ ]:
# !pip install -U 'spacy'

In [ ]:
!python -m spacy download en_core_web_lg

2023-10-04 18:43:47.699497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
# input file paths
input_file_path1 = '/content/Masked_Text_Only_Test.json'
input_text = []

# open the JSON file containing a dictionary of {'text':'Given text...'}
with open(input_file_path1, 'r', encoding='utf-8') as inp_txt_file:
    for line in inp_txt_file:
        # parse the lines as JSON objects
        data = json.loads(line)
        input_text.append(data['text'])

print(len(input_text))

90


In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "Universal-NER/UniNER-7B-all"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    max_length=3000,
    do_sample=True,
    top_k=1,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,

)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
temp = 0.7
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':temp})

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              A virtual assistant answers questions from a user based on the provided text.
              USER: Text: {input_text}
              ASSISTANT: I’ve read this text.
              USER: What describes {entity_type} in the text?
              ASSISTANT:
           """

prompt = PromptTemplate(template=template, input_variables=["input_text","entity_type"])

llm_chain = LLMChain(prompt=prompt, llm=llm)


In [ ]:
# text = "??????? - Abdominal aortic aneurysm - BD50.4Z\nAneurysms can develop anywhere along the aorta, but most aortic aneurysms occur in the part of the aorta that's in the belly area (abdomen). Several things can play a role in the development of an abdominal aortic aneurysm, including: Hardening of the arteries (atherosclerosis). Atherosclerosis occurs when fat and other substances build up on the lining of a blood vessel. High blood pressure. High blood pressure can damage and weaken the aorta's walls. Blood vessel diseases. These are diseases that cause blood vessels to become inflamed. Infection in the aorta. Rarely, a bacterial or fungal infection might cause an abdominal aortic aneurysms. Trauma. For example, being injured in a car accident can cause an abdominal aortic aneurysms. "

ENTITY_LABELS = ['Cause', 'Anatomy', 'Composition', 'Diagnosis', 'Surgery', 'Disease', 'Medicine', 'Precaution', 'Riskfactor', 'Symptom', 'Complication']

entities_all = []
entity_types_all = []


for doc_no, text in enumerate(input_text):
    pred_entities = []
    pred_types = []
    for entity_type in ENTITY_LABELS:
        pred = llm_chain.run({"input_text":text,"entity_type":entity_type})
        try:
          pred = json.loads(pred)
          pred_entities.extend(pred)
          pred_types.extend([entity_type+'_E']*len(pred))
        except:
          print(f'JSON error in Doc = {doc_no}')

    # pushing everything into a list of list - these will store all the predictions for the whole test set
    entities_all.append(pred_entities)
    entity_types_all.append(pred_types)


In [ ]:
# showing a single output from the UniversalNER
print(len(entities_all[0]) == len(entity_types_all[0]))
print()
print(f'entities = {entities_all[0]}')
print(f'entity_types = {entity_types_all[0]}')
print(len(entities_all))

True

entities = ['Hardening of the arteries', 'atherosclerosis', 'High blood pressure', 'Trauma', 'aorta', 'belly area', 'abdomen', 'Abdominal aortic aneurysm', 'Aneurysms', 'Hardening of the arteries', 'atherosclerosis', 'High blood pressure', 'Blood vessel diseases', 'Infection in the aorta', 'Trauma', 'High blood pressure', 'Hardening of the arteries', 'atherosclerosis', 'High blood pressure', 'damage', 'weaken', 'inflamed', 'Infection', 'injured', 'car accident']
entity_types = ['Cause_E', 'Cause_E', 'Cause_E', 'Cause_E', 'Anatomy_E', 'Anatomy_E', 'Anatomy_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Riskfactor_E', 'Complication_E', 'Complication_E', 'Complication_E', 'Complication_E', 'Complication_E', 'Complication_E', 'Complication_E', 'Complication_E', 'Complication_E']
90


In [ ]:
# text = "??????? - Abdominal aortic aneurysm - BD50.4Z\nAneurysms can develop anywhere along the aorta, but most aortic aneurysms occur in the part of the aorta that's in the belly area (abdomen). Several things can play a role in the development of an abdominal aortic aneurysm, including: Hardening of the arteries (atherosclerosis). Atherosclerosis occurs when fat and other substances build up on the lining of a blood vessel. High blood pressure. High blood pressure can damage and weaken the aorta's walls. Blood vessel diseases. These are diseases that cause blood vessels to become inflamed. Infection in the aorta. Rarely, a bacterial or fungal infection might cause an abdominal aortic aneurysms. Trauma. For example, being injured in a car accident can cause an abdominal aortic aneurysms. "

# entities = ['Hardening of the arteries', 'atherosclerosis', 'High blood pressure', 'Trauma', 'aorta', 'belly area', 'abdomen', 'blood vessel', "aorta's walls", 'Abdominal aortic aneurysm', 'Aneurysms', 'Hardening of the arteries', 'atherosclerosis', 'High blood pressure', 'Blood vessel diseases', 'Infection', 'Trauma', 'Hardening of the arteries', 'High blood pressure']
# entity_types = ['Cause_E', 'Cause_E', 'Cause_E', 'Cause_E', 'Anatomy_E', 'Anatomy_E', 'Anatomy_E', 'Anatomy_E', 'Anatomy_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Disease_E', 'Riskfactor_E', 'Riskfactor_E']

In [ ]:
def find_entity_positions_UniNER(text, entities, entity_types):

    # Initializing a dictionary to store the first occurrence of each entity type for each entity
    entity_dict = {}
    for entity, entity_type in zip(entities, entity_types):
        if entity not in entity_dict:
            entity_dict[entity] = entity_type

    # List to store the entity indices along with their types
    entity_indices = []
    new_entity_types = []

    for entity, entity_type in entity_dict.items():
        start_index = 0
        while True:
            start_index = text.find(entity, start_index)
            if start_index == -1:
                break
            end_index = start_index + len(entity)
            entity_indices.append([start_index, end_index])
            new_entity_types.append(entity_type)
            start_index = end_index  # Move to the next position for the next search

    # Displaying the result
    # print(len(entity_indices))
    # print(entity_indices)
    return entity_indices, new_entity_types

In [ ]:
def get_token_span_from_char_span(doc, char_start, char_end):
    start_token_index = None
    end_token_index = None

    for token in doc:
        if start_token_index is None and token.idx >= char_start:
            start_token_index = token.i
        if token.idx + len(token) > char_end:
            end_token_index = token.i
            break

    if start_token_index is not None and end_token_index is not None:
        return (start_token_index, end_token_index)
    else:
        return None

In [ ]:
def construct_json(text, entity_positions, entity_types):
    '''This function will construct the JSON file for evaluation and also generate span cat'''
    entities_span = []
    doc_span = []
    orig_doc = nlp(text)

    for ent_pos, ent_type in zip(entity_positions, entity_types):
        # This will be used for SemEval Evaluation
        entities_span.append((ent_pos[0], ent_pos[1], ent_type))

        # This will be used to visualize the Span
        token_span = get_token_span_from_char_span(orig_doc, ent_pos[0], ent_pos[1])
        if token_span is not None:
            doc_span.append(Span(orig_doc, token_span[0], token_span[1], ent_type))

    # A dictionary in the format {'text': 'Tuberculosis generally damages the lungs', 'entities': [(0, 12, 'Disease_E'), (35, 40, 'Anatomy_E')]}
    ner_prediction = {'text': text, 'entities': entities_span}
    orig_doc.spans["sc"] = doc_span

    return orig_doc, ner_prediction

In [ ]:
def save_predictions(ner_predictions, filename, semeval_format=True):
  # Saving the predictions as JSON - each dictionary on a line
  semeval_ent = []
  with open(filename, 'w', encoding='UTF-8') as json_file:
    for pred in ner_predictions:
      if semeval_format:
        # prodigy format to work with nereval library - for SemEval 2013 - 9.1 task.
        tmp_ent = []
        for ent in pred['entities']:
          # saved in this format: [{"label": "PER", "start": 2, "end": 4}, ... ]
          tmp_ent.append({"label": ent[2], "start": ent[0], "end": ent[1]})

        semeval_ent.append(tmp_ent)

      else:
        # this is regullar spacy format, can be used for spacy's default evaluation later
        json_file.write(json.dumps(pred, ensure_ascii=False))
        json_file.write('\n')

    if semeval_format:
      # dumping it into a JSON file
      json_file.write(json.dumps(semeval_ent, ensure_ascii=False))

  return semeval_ent
  # # This is single line JSON Dump of the entile list of dictionary - parser cannot parse it directly
  # with open(OUTPUT_DIR+'/predition.jsonl', 'w') as fout:
  #     json.dump(ner_predictions, fout)

In [ ]:
'''Assigns different colors to the Entities during visualization.'''

color_list = ['yellow', 'red', 'orange', '#0D9CB4', '#5813C7', '#0D350E', '#1AA436',
          '#1AE0F9', '#BADCA1', '#78A2E5', '#D845FB', '#54B69E']

colors = dict(zip(ENTITY_LABELS, color_list))
options = {"colors": colors}

In [ ]:
def render_pred_span_cat(doc):
  '''Desired Format: doc.spans["sc"] = [Span(doc, 3, 6, "ORG"), Span(doc, 5, 6, "GPE")]
  '''
  spacy.displacy.render(doc, style="span", options=options, jupyter=True)

In [ ]:
ner_predictions_all = []
for text, entities, entity_types in zip(input_text, entities_all, entity_types_all):
    entity_positions, entity_types = find_entity_positions_UniNER(text, entities, entity_types)
    doc, ner_pred_tmp = construct_json(text, entity_positions, entity_types)
    ner_predictions_all.append(ner_pred_tmp)

    _ = save_predictions(ner_predictions_all, filename=f'/content/UniNER_SemEval_Pred_{temp}.jsonl')
    # Saving this for Experiment 2... Spacy Format
    _ = save_predictions(ner_predictions_all, filename=f'/content/UniNER_Spacy_Pred_{temp}.jsonl', semeval_format=False)

    # print(entity_positions)
    # print(entity_types)
    # print()

    # render_pred_span_cat(doc)
    # print()

In [ ]:
from google.colab import files
files.download(f'/content/UniNER_SemEval_Pred_{temp}.jsonl')
files.download(f'/content/UniNER_Spacy_Pred_{temp}.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>